# Lines

## Day 2 - 30 Day Map Challenge

For this map I wanted to demonstrate how to use the [Wherobots Open Data Catalog](https://docs.wherobots.services/1.2.0/tutorials/opendata/introduction/#open-data-catalog) with [SedonaDB](https://wherobots.com/sedona-db/) to use the [Overture Maps transportation dataset](https://medium.com/@dr.jiayu/harnessing-overture-maps-data-apache-sedonas-journey-from-parquet-to-geoparquet-d99f7767a499). Specifically we'll look at roads under construction throughout the world.


![](../img/02-lines-final.png)

You can follow along by creating a free account on [Wherobots Cloud.](https://www.wherobots.services)

In [ ]:
from sedona.spark import *

In [ ]:
config = SedonaContext.builder(). \
    config("spark.sql.catalog.wherobots_examples.type", "hadoop"). \
    config("spark.sql.catalog.wherobots_examples", "org.apache.iceberg.spark.SparkCatalog"). \
    config("spark.sql.catalog.wherobots_examples.warehouse", "s3://wherobots-examples-prod/havasu/warehouse"). \
    config("spark.sql.catalog.wherobots_examples.io-impl", "org.apache.iceberg.aws.s3.S3FileIO"). \
    getOrCreate()

sedona = SedonaContext.create(config)

In [ ]:
sedona.table("wherobots_examples.overture.transportation_segment").printSchema()

```
root
 |-- id: string (nullable = true)
 |-- updatetime: timestamp (nullable = true)
 |-- version: integer (nullable = true)
 |-- level: integer (nullable = true)
 |-- subtype: string (nullable = true)
 |-- connectors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- road: string (nullable = true)
 |-- sources: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- bbox: struct (nullable = true)
 |    |-- minx: double (nullable = true)
 |    |-- maxx: double (nullable = true)
 |    |-- miny: double (nullable = true)
 |    |-- maxy: double (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- geohash: string (nullable = true)
```

Learn more about Wherobots open data in the [Wherobots documentation.](https://docs.wherobots.services/1.2.0/tutorials/opendata/introduction/)

In [ ]:
sedona.table("wherobots_examples.overture.transportation_segment").createOrReplaceTempView("segments")

In [ ]:
segment_df = sedona.sql("""
    SELECT *
    FROM segments
    WHERE road LIKE '%isUnderConstruction%'
""")

In [ ]:
segment_df.count()

```
393894
```

In [ ]:
SedonaKepler.create_map(df=segment_df, name="Under Construction")

![](../img/02-world-view.png)